In [ ]:
from configuration import configuration;
from trajectory import trajectory;
from step import environment;
import numpy as np;
import multiprocessing;
import json;

from model import descriptorNN;
import torch;
from train import PPO;
from action_space import actions;

n_traj = 10;
conf = configuration();
conf.load('POSCARs/CONTCAR_H_Cu1');
conf.set_potential();
env = environment(conf, max_iter=30);
env.relax(accuracy = 0.05)
print(conf.potential())
conf.plot_configuration()
model = descriptorNN();
trainer = PPO(model);
traj = trajectory(1,0);

horizon = 100;
for epoch in range(horizon):
    action_space = actions(conf);
    act_id, act_probs = trainer.select_action(conf.atoms,action_space);
    action = action_space[act_id];
    info = {'act':act_id, 'act_probs':act_probs.tolist(),'act_space':action_space,'state':conf.atoms.copy(),
            'E_min':conf.potential()};

    E_next, fail = env.step(action, accuracy = 0.02);
    if(not fail):
        E_s, fail = env.saddle();
        info['E_s'] = E_s;

    info['next'], info['fail'], info['E_next'] =conf.atoms.copy(), fail, E_next;
    traj.add(info);
    trainer.update(traj,0.9)